In [33]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, TimeDistributed, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
import pandas as pd

In [34]:
train_data = pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_train.csv")
test_data = pd.read_csv("/kaggle/input/sign-language-mnist/sign_mnist_test.csv")

In [37]:
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values


X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

In [39]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [14]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [40]:
y_train = to_categorical(y_train, 25)
y_test = to_categorical(y_test, 25)

In [41]:
num_frames_per_sequence = 10

In [42]:
X_train = X_train[:X_train.shape[0] - (X_train.shape[0] % num_frames_per_sequence)]
X_test = X_test[:X_test.shape[0] - (X_test.shape[0] % num_frames_per_sequence)]

In [44]:
X_train = X_train.reshape(-1, num_frames_per_sequence, 28, 28, 1)
X_test = X_test.reshape(-1, num_frames_per_sequence, 28, 28, 1)

In [45]:
y_train = y_train[:X_train.shape[0]]
y_test = y_test[:X_test.shape[0]]

In [49]:
model = Sequential([
    TimeDistributed(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1))),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
    TimeDistributed(Conv2D(64, (3, 3), activation='relu')),
    TimeDistributed(MaxPooling2D(pool_size=(2, 2))),
    TimeDistributed(Flatten()),
    LSTM(128),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(25, activation='softmax')  # 25 تصنيف (A إلى Y)
])

In [52]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [54]:
model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 35s 350ms/step - accuracy: 0.0347 - loss: 3.2786 - val_accuracy: 0.0237 - val_loss: 3.2134
Epoch 2/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 30s 348ms/step - accuracy: 0.0484 - loss: 3.2093 - val_accuracy: 0.0237 - val_loss: 3.2025
Epoch 3/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 31s 355ms/step - accuracy: 0.0358 - loss: 3.2027 - val_accuracy: 0.0432 - val_loss: 3.2018
Epoch 4/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 30s 349ms/step - accuracy: 0.0548 - loss: 3.1885 - val_accuracy: 0.0432 - val_loss: 3.2072
Epoch 5/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 27s 318ms/step - accuracy: 0.0467 - loss: 3.1874 - val_accuracy: 0.0237 - val_loss: 3.2064
Epoch 6/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 30s 344ms/step - accuracy: 0.0473 - loss: 3.1901 - val_accuracy: 0.0181 - val_loss: 3.2064
Epoch 7/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 30s 348ms/step - accuracy: 0.0556 - loss: 3.1839 - val_accuracy: 0.0237 - val_loss: 3.2049
Epoch 8/15
86/86 ━━━━━━━━━━━━━━━━━━━━ 30s 350ms/step - accuracy: 0.0468 - loss: 3.1862 - val_accu

In [55]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.0174 - loss: 3.1999
Test Accuracy: 0.02


In [56]:
model.save("sign_language_mnist_model.h5")